------------
# Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras

import tensorflow as tf

from keras import models, layers

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
config = tf.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11777514172561118115
]


---------------
# Loading Data into System

In [5]:
x_lstm_train = np.load(os.path.join('data', 'x_lstm_train.npy'))
x_lstm_test = np.load(os.path.join('data', 'x_lstm_test.npy'))

x_static_train = np.load(os.path.join('data', 'x_static_train.npy'))
x_static_test = np.load(os.path.join('data', 'x_static_test.npy'))

y_train = np.load(os.path.join('data', 'y_train.npy'))
y_test = np.load(os.path.join('data', 'y_test.npy'))

In [6]:
print(x_lstm_train.shape, x_lstm_test.shape)
print(x_static_train.shape, x_static_test.shape)
print(y_train.shape, y_test.shape)

(31857, 48) (7965, 48)
(31857, 12) (7965, 12)
(31857,) (7965,)


# Re-Structuring the data

In [7]:
x_lstm_train = np.expand_dims(x_lstm_train, axis=-1)
x_lstm_test = np.expand_dims(x_lstm_test, axis=-1)

x_static_train = np.expand_dims(x_static_train, axis=-1)
x_static_test = np.expand_dims(x_static_test, axis=-1)

y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

In [8]:
print(x_lstm_train.shape, x_lstm_test.shape)
print(x_static_train.shape, x_static_test.shape)
print(y_train.shape, y_test.shape)

(31857, 48, 1) (7965, 48, 1)
(31857, 12, 1) (7965, 12, 1)
(31857, 1) (7965, 1)


# Model Creation

In [9]:
# 32, 16, 30 
model = keras.Sequential()

lstm_input = layers.Input(shape=(x_lstm_train.shape[1], x_lstm_train.shape[2]))
static_input = layers.Input(shape=(x_static_train.shape[1], x_static_train.shape[2]))

lstm_dense = layers.Dense(2)(lstm_input)

lstm_stack_1, lstm_stack_1_c = layers.GRU(72, return_sequences=True, return_state=True)(lstm_dense)
drop_1 = layers.Dropout(0.05)(lstm_stack_1)

lstm_stack_2, lstm_stack_2_c = layers.GRU(36, return_sequences=True, return_state=True)(drop_1)


static_dense = layers.Dense(2)(static_input)
static_flatten = layers.Flatten()(static_dense)

concatenate_lstm_static = layers.Concatenate(axis=-1)([lstm_stack_1_c, lstm_stack_2_c, static_flatten])

drop_2 = layers.Dropout(0.1)(concatenate_lstm_static)
dense_1 = layers.Dense(32)(drop_2)
drop_3 = layers.Dropout(0.05)(dense_1)
dense_2 = layers.Dense(8)(drop_3)
drop_4 = layers.Dropout(0.02)(dense_2)

output = layers.Dense(1)(drop_4)


model = models.Model(inputs=[lstm_input, static_input], outputs=output)
model.summary()
model.compile(
    loss='mse', 
    optimizer='adam', 
    metrics=['mse','mae']
)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 1)        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 48, 2)        4           input_1[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, 48, 72), (No 16200       dense_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 12, 1)        0                  

In [ ]:
history = model.fit(
    [x_lstm_train, x_static_train], y_train,
    epochs=20,
    batch_size=48,
    verbose=1,
    validation_data=([x_lstm_test, x_static_test], y_test),
    shuffle=True
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 31857 samples, validate on 7965 samples
Epoch 1/20




23328/31857 [====================>.........] - ETA: 8s - loss: 157.9050 - mean_squared_error: 157.9050 - mean_absolute_error: 7.7629